In [5]:
import pandas as pd

In [6]:
cols = pd.read_csv("../Resources/Columns.csv")
data = pd.read_csv("../Resources/nhes_19_pfi_v1_0.csv")

col_list = []

In [7]:
for col, row in cols.iteritems():
    col_list = row.values

col_list

array(['BASMID', 'ALLGRADEX', 'SEGRADES', 'CENREG', 'EDCPUB', 'EDCCAT',
       'EDCREL', 'EDCPRI', 'EDCINTK12', 'EDCHSFL', 'DISTASSI',
       'SCHRTSCHL', 'SCHLMAGNET', 'SPBSCH ', 'SOTHRSCH ', 'STUTR ',
       'SOTHSCH ', 'SEENJOY', 'SEABSNT', 'FCSCHOOL', 'FCTEACHR', 'FCSTDS',
       'FCSUPPRT ', 'FHHOME', 'FHWKHRS', 'FOSTORY2X', 'FOCRAFTS',
       'FOGAMES', 'FOBUILDX', 'FOSPORT', 'FORESPON', 'FOHISTX',
       'FODINNERX', 'FOLIBRAYX', 'FOBOOKSTX', 'FOCONCRTX', 'FOMUSEUMX',
       'FOZOOX', 'FOGROUPX', 'FOSPRTEVX ', 'HHENGLISH', 'CSPEAKX',
       'HHTOTALXX', 'HHPRTNRSX', 'P1REL', 'P1SEX', 'P1MRSTA ', 'P1AGE',
       'P2GUARD ', 'P2REL', 'P2SEX', 'P2MRSTA', 'PAR1EMPL', 'PAR1FTFY',
       'PAR1MARST', 'PAR2FTFY', 'PAR2MARST', 'NUMSIBSX', 'TTLHHINC',
       'OWNRNTHB', 'HVINTSPHO', 'HVINTCOM', 'INTACC', 'CHLDNT', 'LRNCOMP',
       'LRNTAB', 'LRNCELL'], dtype=object)

In [8]:
new_df = data.filter(col_list)
new_df

,BASMID,ALLGRADEX,SEGRADES,CENREG,EDCPUB,EDCCAT,EDCREL,EDCPRI,EDCINTK12,EDCHSFL,...,NUMSIBSX,TTLHHINC,OWNRNTHB,HVINTSPHO,HVINTCOM,INTACC,CHLDNT,LRNCOMP,LRNTAB,LRNCELL
0,20191000012,12,1,4,1,2,2,2,2,2,...,1,8,2,1,1,1,1,1,2,1
1,20191000029,4,5,4,1,2,2,2,2,2,...,1,5,2,1,1,1,1,1,1,1
2,20191000059,11,3,2,2,2,1,2,2,2,...,1,3,2,1,2,3,4,1,2,1
3,20191000070,6,1,2,1,2,2,2,2,2,...,1,2,1,1,1,1,2,1,1,1
4,20191000078,9,1,2,1,2,2,2,2,2,...,1,10,1,1,1,1,2,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16441,20191225472,7,1,3,1,2,2,2,2,2,...,2,9,1,1,1,1,2,1,1,1
16442,20191225475,2,5,2,1,2,2,2,2,2,...,3,9,1,1,1,1,1,2,1,2
16443,20191225477,12,1,1,1,2,2,2,2,2,...,2,2,2,1,1,1,1,1,2,1
16444,20191225479,13,1,4,1,2,2,2,2,2,...,0,3,2,1,2,3,2,2,2,1


In [9]:
values = [-1, -9]
columns = list(new_df)
print(columns)

for col in columns:
    new_df = new_df[new_df[col].isin(values) == False]

new_df

['BASMID', 'ALLGRADEX', 'SEGRADES', 'CENREG', 'EDCPUB', 'EDCCAT', 'EDCREL', 'EDCPRI', 'EDCINTK12', 'EDCHSFL', 'DISTASSI', 'SCHRTSCHL', 'SCHLMAGNET', 'SEENJOY', 'SEABSNT', 'FCSCHOOL', 'FCTEACHR', 'FCSTDS', 'FHHOME', 'FHWKHRS', 'FOSTORY2X', 'FOCRAFTS', 'FOGAMES', 'FOBUILDX', 'FOSPORT', 'FORESPON', 'FOHISTX', 'FODINNERX', 'FOLIBRAYX', 'FOBOOKSTX', 'FOCONCRTX', 'FOMUSEUMX', 'FOZOOX', 'FOGROUPX', 'HHENGLISH', 'CSPEAKX', 'HHTOTALXX', 'HHPRTNRSX', 'P1REL', 'P1SEX', 'P1AGE', 'P2REL', 'P2SEX', 'P2MRSTA', 'PAR1EMPL', 'PAR1FTFY', 'PAR1MARST', 'PAR2FTFY', 'PAR2MARST', 'NUMSIBSX', 'TTLHHINC', 'OWNRNTHB', 'HVINTSPHO', 'HVINTCOM', 'INTACC', 'CHLDNT', 'LRNCOMP', 'LRNTAB', 'LRNCELL']


,BASMID,ALLGRADEX,SEGRADES,CENREG,EDCPUB,EDCCAT,EDCREL,EDCPRI,EDCINTK12,EDCHSFL,...,NUMSIBSX,TTLHHINC,OWNRNTHB,HVINTSPHO,HVINTCOM,INTACC,CHLDNT,LRNCOMP,LRNTAB,LRNCELL
0,20191000012,12,1,4,1,2,2,2,2,2,...,1,8,2,1,1,1,1,1,2,1
4,20191000078,9,1,2,1,2,2,2,2,2,...,1,10,1,1,1,1,2,1,2,2
5,20191000083,6,5,4,1,2,2,2,2,2,...,0,10,1,1,1,1,2,1,1,1
7,20191000149,8,1,2,1,2,2,2,2,2,...,2,3,1,1,1,1,1,1,1,1
8,20191000163,5,5,3,1,2,2,2,2,2,...,1,12,1,1,1,1,2,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16439,20191225464,14,1,1,1,2,2,2,2,2,...,2,4,2,1,1,1,1,1,1,1
16441,20191225472,7,1,3,1,2,2,2,2,2,...,2,9,1,1,1,1,2,1,1,1
16442,20191225475,2,5,2,1,2,2,2,2,2,...,3,9,1,1,1,1,1,2,1,2
16443,20191225477,12,1,1,1,2,2,2,2,2,...,2,2,2,1,1,1,1,1,2,1


In [10]:
count = new_df[new_df.columns[1:]].eq(-1).sum().to_frame().T
for col, row in count.iteritems():
    print(col)
    print(row.values)

ALLGRADEX
[0]
SEGRADES
[0]
CENREG
[0]
EDCPUB
[0]
EDCCAT
[0]
EDCREL
[0]
EDCPRI
[0]
EDCINTK12
[0]
EDCHSFL
[0]
DISTASSI
[0]
SCHRTSCHL
[0]
SCHLMAGNET
[0]
SEENJOY
[0]
SEABSNT
[0]
FCSCHOOL
[0]
FCTEACHR
[0]
FCSTDS
[0]
FHHOME
[0]
FHWKHRS
[0]
FOSTORY2X
[0]
FOCRAFTS
[0]
FOGAMES
[0]
FOBUILDX
[0]
FOSPORT
[0]
FORESPON
[0]
FOHISTX
[0]
FODINNERX
[0]
FOLIBRAYX
[0]
FOBOOKSTX
[0]
FOCONCRTX
[0]
FOMUSEUMX
[0]
FOZOOX
[0]
FOGROUPX
[0]
HHENGLISH
[0]
CSPEAKX
[0]
HHTOTALXX
[0]
HHPRTNRSX
[0]
P1REL
[0]
P1SEX
[0]
P1AGE
[0]
P2REL
[0]
P2SEX
[0]
P2MRSTA
[0]
PAR1EMPL
[0]
PAR1FTFY
[0]
PAR1MARST
[0]
PAR2FTFY
[0]
PAR2MARST
[0]
NUMSIBSX
[0]
TTLHHINC
[0]
OWNRNTHB
[0]
HVINTSPHO
[0]
HVINTCOM
[0]
INTACC
[0]
CHLDNT
[0]
LRNCOMP
[0]
LRNTAB
[0]
LRNCELL
[0]


In [11]:
len(new_df)

9396

In [12]:
new_df.to_csv("../Resources/nhes_19_pfi_cleaned.csv")